## Importing Libraries

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

**2175. The Change in Global Rankings (Medium)**

**Table: TeamPoints**

| Column Name | Type    |
|-------------|---------|
| team_id     | int     |
| name        | varchar |
| points      | int     |

team_id contains unique values.
Each row of this table contains the ID of a national team, the name of the country it represents, and the points it has in the global rankings. No two teams will represent the same country.
 
**Table: PointsChange**

| Column Name   | Type |
|---------------|------|
| team_id       | int  |
| points_change | int  |

team_id contains unique values.
Each row of this table contains the ID of a national team and the change in its points in the global rankings.
points_change can be:
- 0: indicates no change in points.
- positive: indicates an increase in points.
- negative: indicates a decrease in points.

Each team_id that appears in TeamPoints will also appear in this table.
 
The global ranking of a national team is its rank after sorting all the teams by their points in descending order. If two teams have the same points, we break the tie by sorting them by their name in lexicographical order.

The points of each national team should be updated based on its corresponding points_change value.

**Write a solution to calculate the change in the global rankings after updating each team's points.**

Return the result table in any order.

The result format is in the following example.

**Example 1:**

**Input:** 

**TeamPoints table:**

| team_id | name        | points |
|---------|-------------|--------|
| 3       | Algeria     | 1431   |
| 1       | Senegal     | 2132   |
| 2       | New Zealand | 1402   |
| 4       | Croatia     | 1817   |

**PointsChange table:**
| team_id | points_change |
|---------|---------------|
| 3       | 399           |
| 2       | 0             |
| 4       | 13            |
| 1       | -22           |

**Output:** 
| team_id | name        | rank_diff |
|---------|-------------|-----------|
| 1       | Senegal     | 0         |
| 4       | Croatia     | -1        |
| 3       | Algeria     | 1         |
| 2       | New Zealand | 0         |

**Explanation:** 
The global rankings were as follows:
| team_id | name        | points | rank |
|---------|-------------|--------|------|
| 1       | Senegal     | 2132   | 1    |
| 4       | Croatia     | 1817   | 2    |
| 3       | Algeria     | 1431   | 3    |
| 2       | New Zealand | 1402   | 4    |

After updating the points of each team, the rankings became the following:
| team_id | name        | points | rank |
|---------|-------------|--------|------|
| 1       | Senegal     | 2110   | 1    |
| 3       | Algeria     | 1830   | 2    |
| 4       | Croatia     | 1830   | 3    |
| 2       | New Zealand | 1402   | 4    |

- Since after updating the points Algeria and Croatia have the same points, they are ranked according to their lexicographic order.
- Senegal lost 22 points but their rank did not change.
- Croatia gained 13 points but their rank decreased by one.
- Algeria gained 399 points and their rank increased by one.
- New Zealand did not gain or lose points and their rank did not change.

In [0]:
team_points_data_2175 = [
    (3, "Algeria", 1431),
    (1, "Senegal", 2132),
    (2, "New Zealand", 1402),
    (4, "Croatia", 1817),
]

team_points_columns_2175 = ["team_id", "name", "points"]
df_teams_2175 = spark.createDataFrame(team_points_data_2175, team_points_columns_2175)
df_teams_2175.show()

points_change_data_2175 = [
    (3, 399),
    (2, 0),
    (4, 13),
    (1, -22),
]

points_change_columns_2175 = ["team_id", "points_change"]
df_changes_2175 = spark.createDataFrame(points_change_data_2175, points_change_columns_2175)
df_changes_2175.show()

+-------+-----------+------+
|team_id|       name|points|
+-------+-----------+------+
|      3|    Algeria|  1431|
|      1|    Senegal|  2132|
|      2|New Zealand|  1402|
|      4|    Croatia|  1817|
+-------+-----------+------+

+-------+-------------+
|team_id|points_change|
+-------+-------------+
|      3|          399|
|      2|            0|
|      4|           13|
|      1|          -22|
+-------+-------------+



In [0]:
df_2175 = df_teams_2175.join(df_changes_2175, "team_id")

In [0]:
w_rank = Window.orderBy(desc("points"), "name")

In [0]:
df_2175 = df_2175\
            .withColumn("original_rank", rank().over(w_rank))\
                .withColumn("updated_points", col("points") + col("points_change"))

/databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/expressions.py:1017: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


In [0]:
w_rank_new = Window.orderBy(desc("updated_points"), "name")

In [0]:
df_2175 = df_2175\
            .withColumn("new_rank", rank().over(w_rank_new))

/databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/expressions.py:1017: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


In [0]:
df_2175\
    .select( "team_id", "name", (col("original_rank") - col("new_rank")).alias("rank_diff")).show()

/databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/expressions.py:1017: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


+-------+-----------+---------+
|team_id|       name|rank_diff|
+-------+-----------+---------+
|      1|    Senegal|        0|
|      3|    Algeria|        1|
|      4|    Croatia|       -1|
|      2|New Zealand|        0|
+-------+-----------+---------+

